## Conversational Memory for LLMs
Q: Do the gpt models from OpenAI retain memory of previous prompts and incorporate them into subsequent responses?

A: Simply put, the gpt model doesn't remember past prompts. Each time you interact with it, you have to provide all the necessary information again. Currently, all large language models work this way—they need the full set of instructions every time.

In [3]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

### Without conversation history/memory: 

This helper function will make it easier to use prompts and look at the generated outputs. The problem with this helper is that every time we make a request, the model does not remember the previous requests.

In [12]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

In [13]:
print("Response: ", get_completion("My name is Aaron"))

Response:  Nice to meet you, Aaron! How can I assist you today?


In [8]:
print("Response: " , get_completion("What's my name?"))

Response:  I'm sorry, I don't have access to that information.


### With conversation history/memory: 
Modify the helper function to help the model remember previous prompts and responses:

In [15]:
def get_completion(messages, model="gpt-3.5-turbo"):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    # Append AI response to the messages list
    messages.append({
        "role": "system",
        "content": response.choices[0].message.content
    })
    return response.choices[0].message.content

messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant."
    }
]

When OpenAI replies, we add that to messages array:

In [16]:
# Example usage
prompt = "My name is Aaron"
messages.append({"role": "user", "content": prompt})

print("Prompt:", prompt)
print("Response:", get_completion(messages))


Prompt: My name is Aaron
Response: Nice to meet you, Aaron! How can I assist you today?


In [17]:
print(json.dumps(messages, indent=4))

[
    {
        "role": "system",
        "content": "You are a helpful assistant."
    },
    {
        "role": "user",
        "content": "My name is Aaron"
    },
    {
        "role": "system",
        "content": "Nice to meet you, Aaron! How can I assist you today?"
    }
]


When making another request, we send the whole array back:

In [18]:
prompt = "What is my name?"
messages.append({"role": "user", "content": prompt})

print("Prompt:", prompt)
print("Response:", get_completion(messages))

Prompt: What is my name?
Response: Your name is Aaron.


In [45]:
print(json.dumps(messages, indent=4))

[
    {
        "role": "system",
        "content": "You are a helpful assistant."
    },
    {
        "role": "user",
        "content": "My name is Aaron"
    },
    {
        "role": "system",
        "content": "Nice to meet you, Aaron! How can I assist you today?"
    },
    {
        "role": "user",
        "content": "What is my name?"
    },
    {
        "role": "system",
        "content": "Your name is Aaron."
    }
]
